In [1]:
import os
os.environ['SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL'] = 'True'

In [ ]:
!git clone https://github.com/jianhuupenn/ItClust/tree/master/tutorial/data/pancreas.git
pip install keras==2.2.4 tensorflow==1.14.0
pip show anndata numpy pandas tensorflow keras scipy scanpy natsort sklearn

In [5]:
import ItClust as ic
import scanpy.api as sc
import os
from numpy.random import seed
from tensorflow import set_random_seed
import pandas as pd
import numpy as np
import warnings
os.environ["CUDA_VISIBLE_DEVICES"]="1"
warnings.filterwarnings("ignore")
#import sys
#!{sys.executable} -m pip install 'scanpy==1.4.4.post1'
#Set seeds
seed(20180806)
np.random.seed(10)
set_random_seed(20180806) # on GPU may be some other default

Using TensorFlow backend.
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xs333/.conda/envs/ItClu

In [6]:
import warnings
# Suppress specific ImportWarning
warnings.filterwarnings("ignore")

## Import the data

In [11]:
adata = sc.read("./bct_raw.h5ad")
adata.obs.rename(columns={'BATCH':'batch'}, inplace=True)
adata

AnnData object with n_obs × n_vars = 9288 × 1222 
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'study', 'cell.class', 'library_size', 'detected_genes', 'batch', 'celltype'
    var: 'gene.symbols', 'gene.counts'

In [ ]:
adata.obs = pd.read_csv("./labels_for_ItClust/randomly_wrong_30_bct.csv", index_col=0)

In [13]:
adata

AnnData object with n_obs × n_vars = 9288 × 1222 
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'study', 'cell.class', 'library_size', 'detected_genes', 'batch', 'cell_type', 'batch_labels'
    var: 'gene.symbols', 'gene.counts'

In [14]:
adata.obs['celltype'] = adata.obs['batch_labels'].values

## Common Code

In [15]:
adata.raw = adata
adata.layers["counts"] = adata.X.copy()

In [16]:
print(adata)

AnnData object with n_obs × n_vars = 9288 × 1222 
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'study', 'cell.class', 'library_size', 'detected_genes', 'batch', 'cell_type', 'batch_labels', 'celltype'
    var: 'gene.symbols', 'gene.counts'
    layers: 'counts'


In [17]:
raw_adata = adata.raw.to_adata()

In [18]:
raw_adata.var_names = [name.decode('utf-8') if isinstance(name, bytes) else name for name in raw_adata.var_names]

In [19]:
raw_adata.obs_names = [name.decode('utf-8') if isinstance(name, bytes) else name for name in raw_adata.obs_names]

In [20]:
raw_adata.var_names

Index(['ENSMUSG00000092341', 'ENSMUSG00000029580', 'ENSMUSG00000023043',
       'ENSMUSG00000064341', 'ENSMUSG00000031765', 'ENSMUSG00000017009',
       'ENSMUSG00000016559', 'ENSMUSG00000064370', 'ENSMUSG00000024661',
       'ENSMUSG00000049382',
       ...
       'ENSMUSG00000036580', 'ENSMUSG00000010663', 'ENSMUSG00000040605',
       'ENSMUSG00000039431', 'ENSMUSG00000028634', 'ENSMUSG00000022218',
       'ENSMUSG00000020593', 'ENSMUSG00000030905', 'ENSMUSG00000022949',
       'ENSMUSG00000028545'],
      dtype='object', length=1222)

In [21]:
raw_adata.obs_names

Index(['vis1', 'vis2', 'vis3', 'vis4', 'vis5', 'vis6', 'vis7', 'vis8', 'vis9',
       'vis10',
       ...
       'wal4363', 'wal4364', 'wal4365', 'wal4366', 'wal4367', 'wal4370',
       'wal4371', 'wal4372', 'wal4374', 'wal4375'],
      dtype='object', length=9288)

In [22]:
raw_adata.raw = raw_adata

In [23]:
adata_train = raw_adata.raw.to_adata()

In [24]:
adata_test = raw_adata.raw.to_adata()

In [25]:
adata_train.raw = adata_train

In [26]:
adata_test.raw = adata_test

In [27]:
print(adata_train.raw.var_names)

Index(['ENSMUSG00000092341', 'ENSMUSG00000029580', 'ENSMUSG00000023043',
       'ENSMUSG00000064341', 'ENSMUSG00000031765', 'ENSMUSG00000017009',
       'ENSMUSG00000016559', 'ENSMUSG00000064370', 'ENSMUSG00000024661',
       'ENSMUSG00000049382',
       ...
       'ENSMUSG00000036580', 'ENSMUSG00000010663', 'ENSMUSG00000040605',
       'ENSMUSG00000039431', 'ENSMUSG00000028634', 'ENSMUSG00000022218',
       'ENSMUSG00000020593', 'ENSMUSG00000030905', 'ENSMUSG00000022949',
       'ENSMUSG00000028545'],
      dtype='object', length=1222)


In [28]:
print(adata_train.raw.obs_names)

Index(['vis1', 'vis2', 'vis3', 'vis4', 'vis5', 'vis6', 'vis7', 'vis8', 'vis9',
       'vis10',
       ...
       'wal4363', 'wal4364', 'wal4365', 'wal4366', 'wal4367', 'wal4370',
       'wal4371', 'wal4372', 'wal4374', 'wal4375'],
      dtype='object', length=9288)


In [29]:
clf=ic.transfer_learning_clf()
clf.fit(adata_train, adata_test)

the var_names of adata.raw: adata.raw.var_names.is_unique=: True
the var_names of adata.raw: adata.raw.var_names.is_unique=: True
Number of genes used:  1222 out of  1222
The number of training celltypes is:  4
Training the source network
The layer numbers are[32, 16]
The shape of xtrain is:9283:1222
The shape of xtest is:9283:1222
Doing DEC: pretrain



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

...Pretraining...
Doing SAE: pretrain_stacks
Pretraining the 1th layer...
learning rate = 0.1



2025-01-08 10:52:25.270887: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2025-01-08 10:52:25.286040: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2800000000 Hz
2025-01-08 10:52:25.286214: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a4ce145a90 executing computations on platform Host. Devices:
2025-01-08 10:52:25.286232: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2025-01-08 10:52:25.356954: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA

learning rate = 0.01
learning rate = 0.001
The 1th layer has been pretrained.
Pretraining the 2th layer...
learning rate = 0.1
learning rate = 0.01
learning rate = 0.001
The 2th layer has been pretrained.
Doing SAE: pretrain_autoencoders
Copying layer-wise pretrained weights to deep autoencoders
Fine-tuning autoencoder end-to-end
learning rate = 0.1
learning rate = 0.010000000000000002
learning rate = 0.001
learning rate = 0.0001
learning rate = 1e-05
learning rate = 1.0000000000000002e-06
Pretraining time:  70.27527904510498
y known, initilize Cluster centroid using y
The shape of cluster_center is (4, 16)
Doing DEC: fit_supervised
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training model finished! Start to fit target network!
Doing DEC: pretrain_transfer
The shape of features is (9283, 16)
The shape of y_trans is (9283,)
...predicted y_test known, use it to get n_cliusters and init_centroid
The length layers  of self.model 4
Doing DE

## Print celltype annotation

In [ ]:
celltype_pred = {}
source_label=pd.Series(clf.adata_train.obs["celltype"], dtype="category")
source_label=source_label.cat.categories.tolist()
num_ori_ct=clf.adata_test.obsm["prob_matrix"+str(clf.save_atr)].shape[1]
target_label=[str(i) for i in range(num_ori_ct)]
for i in range(num_ori_ct):
    end_cell=clf.adata_test.obs.index[clf.adata_test.obs["decisy_trans_True"]==target_label[i]]
    start_cell=clf.adata_test.obs.index[clf.adata_test.obs["trajectory_0"]==target_label[i]]
    overlap=len(set(end_cell).intersection(set(start_cell)))
    celltype_pred[target_label[i]]=[source_label[i], round(overlap/(len(end_cell)+0.0001),3)]

In [ ]:
# bct
print(celltype_pred)

In [ ]:
pred, prob, celltype_pred = clf.predict()
pred.head()

## Save the embeddings

In [30]:
embeddings_dict = {}

In [31]:
embeddings_dict['bct'] = clf.adata_test.obsm["X_Embeded_z"+str(clf.save_atr)]

In [32]:
embeddings_dict['bct']

array([[-0.5774134 , -0.99993765,  0.98875403, ..., -0.99957323,
         0.9984017 , -0.99943304],
       [ 0.79047143, -0.99986094, -0.9995606 , ...,  0.99921536,
         0.99330527, -0.9523891 ],
       [-0.62978166,  0.99999225,  0.9938726 , ..., -0.99952793,
         0.48033828,  0.99958414],
       ...,
       [-0.5983642 , -0.9990979 ,  0.9757828 , ..., -0.9963507 ,
         0.98096764, -0.9871259 ],
       [-0.59169674, -0.9988761 ,  0.9820229 , ..., -0.9990971 ,
         0.9933715 , -0.9965967 ],
       [-0.653225  ,  0.99990714,  0.9904627 , ..., -0.99958956,
         0.49114606,  0.9993787 ]], dtype=float32)

In [ ]:
combined_embeddings = pd.concat(
    {key:pd.DataFrame(value, index=adata_test.obs_names) for key, value in embeddings_dict.items()}, axis=1
)

combined_embeddings.to_csv("./results/randomly_wrong_30_itclust_embeddings_bct.csv")